# Purpose

To reset the `is_eligible_account` status for all players in the `player_game_counts` database.

# Why?

An error in the previous notebook (`15_filter_out_banned_players.ipynb`) caused players to be incorrectly marked as ineligible if the Lichess API call failed for their batch. 

This notebook will set the `is_eligible_account` column to `NULL` for all records, allowing the corrected pipeline to re-process all players accurately.

In [1]:
from pathlib import Path
from utils.database.db_utils import get_db_connection

# --- Database Configuration ---
DB_PATH = (
    Path.cwd().parent
    / "data"
    / "processed"
    / "find_most_active_players"
    / "player_game_counts.duckdb"
)

print(f"Connecting to database at: {DB_PATH}")

# Use a try...finally block to ensure the DB connection is always closed
con = get_db_connection(str(DB_PATH))
try:
    # --- Reset the is_eligible_account column ---
    print("Resetting 'is_eligible_account' column to NULL for all players...")
    
    update_query = "UPDATE player_game_counts SET is_eligible_account = NULL"
    result = con.execute(update_query)
    
    # The result of an UPDATE statement in DuckDB contains the number of affected rows.
    updated_rows = result.fetchone()[0]
    
    print(f"Successfully reset {updated_rows:,} records.")

    # --- Verification Step ---
    print("\nVerifying the reset operation...")
    
    # Check if any records still have a non-NULL value
    remaining_count = con.execute(
        "SELECT COUNT(*) FROM player_game_counts WHERE is_eligible_account IS NOT NULL"
    ).fetchone()[0]

    if remaining_count == 0:
        print("Verification successful: All 'is_eligible_account' values are NULL.")
    else:
        print(f"Verification failed: Found {remaining_count:,} records that were not reset.")

    # Optional: Display a small sample to visually confirm
    print("\nSample of reset data (should all be NULL):")
    sample_df = con.execute("SELECT username, is_eligible_account FROM player_game_counts LIMIT 5").df()
    print(sample_df)

finally:
    con.close()
    print("\nDatabase connection closed.")

Connecting to database at: /Users/a/Documents/personalprojects/chess-opening-recommender/data/processed/find_most_active_players/player_game_counts.duckdb
Resetting 'is_eligible_account' column to NULL for all players...
Successfully reset 2,847,488 records.

Verifying the reset operation...
Verification successful: All 'is_eligible_account' values are NULL.

Sample of reset data (should all be NULL):
               username  is_eligible_account
0         Alien_Project                 <NA>
1  rastujapurcisaabtipe                 <NA>
2           G24_de_tour                 <NA>
3       checkthatmatead                 <NA>
4            ilijabern1                 <NA>

Database connection closed.
